In [ ]:
import numpy as np
import energyflow as ef
import ROOT
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
import time
import os
import itertools
from sklearn.decomposition import PCA
from sklearn import linear_model

### Reading the trees

In [ ]:
f = ROOT.TFile("dataname.root")
t = f.Get("tree")
mc_f = ROOT.TFile("mcname.root")
mc_t = mc_f.Get("tree")

### Loading data and MC jets into lists in order to use the energyflow package

In [ ]:
# jet list of data
events = []
# jet multiplicity of data
jetm = []
# jet pt of data
pttot = []
# for MC
mc_events = []
mc_jetm = []
mc_pttot = []

for e in t:
    if e.jetm > 0:
        n = e.jetm
        if e.pt > 20: 
            ev = 1
            jetm.append(n)
            pttot.append(e.pt)
        else: ev = 0
        ls = []
    elif n >= 0:
        ls.append([e.pt, e.eta, e.phi])
        n -= 1
    if n == 0 and ev == 1:
        events.append(ls)
jetm = np.array(jetm)
pttot = np.array(pttot)

for e in mc_t:
    if e.jetm > 0:
        n = e.jetm
        if e.pt > 20: 
            ev = 1
            mc_jetm.append(n)
            mc_pttot.append(e.pt)
        else: ev = 0
        ls = []
    elif n >= 0:
        ls.append([e.pt, e.eta, e.phi])
        n -= 1
    if n == 0 and ev == 1:
        mc_events.append(ls)
mc_jetm = np.array(mc_jetm)
mc_pttot = np.array(mc_pttot)

### Preparing graphs

In [ ]:
# jet multiplicity lower bound = n for graphs containing n dots (used for excluding jets with jetm < #dots of EFP graph)
jetm_lb = [ 2, 
            2, 3, 
            2, 3, 3, 4, 4, 
            2, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 
            2, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6]
# the 53 EFP multigraphs, each number corresponds to a dot, (a,b) means an edge connects a and b
graphs = [[(0,1)], 
          [(0,1),(0,1)], [(0,1),(0,2)], 
          [(0,1),(0,1),(0,1)], [(0,1),(1,2),(0,2)], [(0,1),(0,1),(0,2)], 
          [(0,1),(0,2),(0,3)], [(0,1),(1,2),(0,3)], 
          [(0,1),(0,1),(0,1),(0,1)], [(0,1),(0,1),(0,2),(0,2)], [(0,1),(1,2),(1,2),(0,2)], 
          [(0,1),(0,1),(0,1),(0,2)], [(0,1),(1,2),(2,3),(3,0)], [(0,1),(0,1),(0,2),(0,3)], 
          [(0,1),(0,1),(0,2),(1,3)], [(0,1),(1,2),(2,3),(3,1)], [(0,1),(0,1),(0,2),(2,3)], 
          [(0,1),(0,2),(0,3),(0,4)], [(0,1),(0,2),(0,3),(1,4)], [(0,1),(0,2),(1,3),(2,4)], 
          [(0,1),(0,1),(0,1),(0,1),(0,1)],
          [(0,1),(0,2),(1,2),(1,2),(1,2)], [(0,1),(0,1),(0,2),(1,2),(1,2)], 
          [(0,1),(0,1),(0,1),(0,1),(0,2)], [(0,1),(0,1),(0,1),(0,2),(0,2)], 
          [(0,1),(0,2),(2,1),(3,1),(3,2)], [(0,1),(0,1),(0,1),(0,3),(0,2)], 
          [(0,1),(0,1),(0,3),(0,2),(0,2)], [(0,1),(0,1),(0,1),(0,2),(1,3)],
          [(0,1),(0,1),(2,1),(2,3),(3,1)], [(0,1),(2,1),(3,1),(3,2),(3,2)],
          [(0,1),(2,1),(3,1),(3,2),(1,2)], [(0,1),(0,2),(0,2),(1,3),(1,3)],
          [(0,1),(2,1),(3,2),(3,0),(1,2)], [(0,1),(0,1),(0,2),(0,2),(1,3)],
          [(0,1),(0,2),(0,2),(0,2),(1,3)], [(0,1),(1,2),(2,3),(3,4),(4,0)],
          [(0,1),(0,1),(0,2),(0,3),(0,4)], [(0,1),(0,2),(1,2),(0,3),(0,4)],
          [(0,1),(0,1),(1,2),(0,3),(0,4)], [(0,1),(1,2),(1,2),(0,3),(0,4)],
          [(0,1),(2,1),(3,1),(3,2),(3,4)], [(0,1),(0,2),(1,3),(3,2),(3,4)],
          [(0,1),(0,2),(1,2),(0,3),(3,4)], [(0,1),(0,2),(0,2),(0,3),(1,4)],
          [(0,1),(0,1),(1,2),(0,3),(3,4)], [(0,1),(1,2),(1,2),(0,3),(3,4)],
          [(0,1),(0,2),(0,3),(0,4),(0,5)], [(0,1),(0,2),(0,3),(1,4),(1,5)],
          [(0,1),(0,2),(0,3),(0,4),(1,5)], [(0,1),(0,2),(1,3),(1,4),(2,5)],
          [(0,1),(0,2),(0,3),(1,4),(2,5)], [(0,1),(1,2),(0,3),(3,4),(4,5)]]

### Calculating EFPs and print out each runtime

##### the results list will contain 53 lists, each will contain the values of all jets of a single EFP

In [ ]:
efps = [ef.EFP(graph, measure='hadr', beta=1, normed=True) for graph in graphs]

In [ ]:
results = []

print("{:15} {:15} {:15}".format('graph_number','degree','runtime(s)'))
for i in range(len(efps)):
    results_tmp = []
    start = time.time()
    for j in range(len(events)):
        results_tmp.append(efps[i].compute(events[j]))
    end = time.time()
    if i<1: deg = 1
    elif i<3: deg = 2
    elif i<8: deg = 3
    elif i<20: deg = 4
    elif i<53: deg = 5
    else: deg = -999
    print("{:<15} {:<15} {:<15.03f}".format(i, deg, end - start))
    np_res_tmp = np.array(results_tmp)
    results.append(np_res_tmp)
results = np.array(results)

In [ ]:
mc_results = []

print("{:15} {:15} {:15}".format('graph_number','degree','runtime(s)'))
for i in range(len(efps)):
    results_tmp = []
    start = time.time()
    for j in range(len(mc_events)):
        results_tmp.append(efps[i].compute(mc_events[j]))
    end = time.time()
    if i<1: deg = 1
    elif i<3: deg = 2
    elif i<8: deg = 3
    elif i<20: deg = 4
    elif i<53: deg = 5
    else: deg = -999
    print("{:<15} {:<15} {:<15.03f}".format(i, deg, end - start))
    np_res_tmp = np.array(results_tmp)
    mc_results.append(np_res_tmp)
mc_results = np.array(mc_results)

### Normalize MC by pt distributioin

In [ ]:
# there will be 200 bins, pt from 20 to 120 GeV
bins = np.linspace(20, 120, 201)
# ..._counts will the a list of bin heights
pt_counts, _ =  np.histogram(pttot, bins)
mcpt_counts, _ = np.histogram(mc_pttot, bins)
# scale_arr is the ratio of the heights
scale_arr = np.array(pt_counts)/np.array(mcpt_counts, float)

In [ ]:
# the weight list contains the weight of each MC jet for filling histograms 
weight = []
for pt in mc_pttot:
    if pt>20 and pt<120:
        weight.append(scale_arr[int(pt*2)-40])
    else:
        weight.append(0)
weight = np.array(weight)

### The original code to draw the EFP distributions

In [ ]:
# the x limit for the histograms
xlim = [0.4, 0.15, 0.1, 0.04, 0.02, 0.03, 0.03, 0.02, 0.02, 0.01,
        0.01, 0.02, 0.01, 0.015, 0.010, 0.01, 0.01, 0.01, 0.01, 0.01,
        0.006, 0.0015, 0.0015, 0.003, 0.003, 0.001, 0.0025, 0.0025, 0.002, 0.001,
        0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.002, 0.001, 0.001,
        0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.002, 0.0005, 0.0005,
        0.0005, 0.0005, 0.0005]

##### for the whole pt range

In [ ]:
for i in range(53):
    directory = 'basis_mc_ptscaled/'
    if not os.path.exists(directory):
        os.makedirs(directory)
    fig, (ax, ax1) = plt.subplots(2,1,gridspec_kw={'height_ratios': [3.5, 1]}, figsize=(6,5))
    bins = np.linspace(0, xlim[i], 21)
    bin_centres = (bins[:-1] + bins[1:])/2
    ax.set_xlim(0, xlim[i])
    ax1.set_xlim(0, xlim[i])
    ax1.set_ylim(0.5,1.5)
    
    plt.xticks(np.arange(0, xlim[i], xlim[i]/5))
    
    
#         for data
    data = results[i][np.logical_and(jetm >= jetm_lb[i], pttot<120)]
    counts, _ = np.histogram(data, bins)
    err = np.sqrt(counts)
    ax.errorbar(bin_centres, counts, yerr=np.sqrt(counts), fmt = 'ok', capsize=3, label = "data")
#         for mc
    mc = mc_results[i][np.logical_and(mc_jetm >= jetm_lb[i], mc_pttot<120)]
    weight_new = weight[np.logical_and(mc_jetm >= jetm_lb[i], mc_pttot<120)]
    mc_counts, _ = np.histogram(mc, bins, weights = weight_new)
    ax.hist(bin_centres, weights = mc_counts, bins = bins, label = "MC")
    ax.axes.get_xaxis().set_visible(False)
    ax.legend(prop={'size': 14}, loc='upper right')
    
    plt.tight_layout()
#     graphs/ contains the multigraphs
    im = plt.imread('graphs/' + str(i) + '.png')
    newax = fig.add_axes([0.76, .6, 0.2, 0.2], zorder=1)
    newax.imshow(im)
    newax.axis('off')
#     ratio
    counts = counts.astype(float)
    ratio = np.divide(counts, mc_counts, out=np.zeros_like(counts), where = mc_counts!=0)
    ratio_err = np.divide(err, mc_counts, out=np.zeros_like(counts), where = mc_counts!=0)
    ax1.errorbar(bin_centres, ratio, yerr = ratio_err, fmt = '.k', capsize=3)
    ax1.set_yticks(np.arange(0.5, 1.5, 0.25), minor=True)
    ax1.grid(color='grey', linestyle='--', linewidth=2, axis='y', alpha=0.7, which='both')
    ax1.set_xlabel('EFP Value', fontsize = 16)
    plt.subplots_adjust(hspace = 0.1)
    plt.gcf().subplots_adjust(bottom=0.12)
    plt.savefig(directory + str(i) + '.png', dpi = 300)

    plt.show()

    plt.close('all')

##### For finer pt ranges

In [ ]:
for j in range(3) :
    for i in range(53):
        directory = 'basis_mc_pt_notptscaled/' + str(j) + '/'
        if not os.path.exists(directory):
            os.makedirs(directory)
        fig, (ax, ax1) = plt.subplots(2,1,gridspec_kw={'height_ratios': [3.5, 1]}, figsize=(6,5))
        lowlim = 20 + 10*j
        uplim = 20 + 10*(j+1)
        
        bins = np.linspace(0, xlim[i]*(1-j*0.2), 11)
        bin_centres = (bins[:-1] + bins[1:])/2
        plt.xlim(0, xlim[i]*(1-j*0.2))
        ax.set_xlim(0, xlim[i]*(1-j*0.2))
        ax1.set_xlim(0, xlim[i]*(1-j*0.2))
        ax1.set_ylim(0.5,1.5)
        plt.xticks(np.arange(0, xlim[i]*(1-j*0.2), xlim[i]*(1-j*0.2)/(5-j)))
        
#         for data
        data = results[i][np.logical_and.reduce((pttot > lowlim, pttot <= uplim, jetm >= jetm_lb[i], pttot<120))]
        counts, _ = np.histogram(data, bins)
        err = np.sqrt(counts)
        ax.errorbar(bin_centres, counts, yerr=np.sqrt(counts), fmt = '.k', capsize=3, label = "data")
#         for mc
        mc = mc_results[i][np.logical_and.reduce((mc_pttot > lowlim, mc_pttot <= uplim, mc_jetm >= jetm_lb[i]))]
        mc_counts, _ = np.histogram(mc, bins)    
        scaled_mc = len(data)/float(len(mc))*mc_counts
        ax.hist(bin_centres, weights = scaled_mc, bins = bins, color = "mediumseagreen", label = "MC")
        ax.axes.get_xaxis().set_visible(False)
        ax.legend(prop={'size': 14}, loc='upper right')
        
        plt.figtext(.96, .75, r'jet_pt$\in$(' + str(lowlim) + ', '+str(uplim)+']', fontsize = 16, ha = 'right')
        plt.tight_layout()

        im = plt.imread('graphs/' + str(i) + '.png')
        newax = fig.add_axes([0.76, .50, 0.2, 0.2], zorder=1)
        newax.imshow(im)
        newax.axis('off')
#         ratio
        counts = counts.astype(float)
        ratio = np.divide(counts, scaled_mc, out=np.zeros_like(counts), where = scaled_mc!=0)
        ratio_err = np.divide(err, scaled_mc, out=np.zeros_like(counts), where = scaled_mc!=0)
        ax1.errorbar(bin_centres, ratio, yerr = ratio_err, fmt = '.k', capsize=3)
        ax1.set_yticks(np.arange(0.5, 1.5, 0.25), minor=True)
        ax1.grid(color='grey', linestyle='--', linewidth=2, axis='y', alpha=0.5, which='both')
        plt.subplots_adjust(hspace = 0.1)
        
        plt.savefig(directory + str(i) + '.png', dpi = 300)

        plt.show()

        plt.close('all')

### Performing PCA

In [ ]:
results_transposed = results.transpose()
for j in range(1,10):
#     j is the dimension of the PCA
    pca = PCA(n_components=j)
#     the results projected onto the PCs (dimension-reduced) represented by the PC basis
    results_transformed = pca.fit_transform(results_transposed)
#     represent the dimension-reduced results by the EFP (original) basis. it's called "centered" because PCA 
#     will automatically center the data first
    centered = np.matmul(results_transformed, pca.components_)
#     un-center, results_approx is the approximated results
    original_transposed = centered + pca.mean_
    results_approx = original_transposed.transpose()
    for i in range(53):
#   put codes to draw the histograms here

### Performing linear regression

#### Standardize the results

In [ ]:
norm_results = np.zeros_like(results)
for i in range(len(results)):
    norm_results[i] = (results[i] - np.mean(results[i]))/np.std(results[i])

#### Mass over pt

In [ ]:
masses = []
i = 0
for e in events:
    px_tot = py_tot = pz_tot = en_tot = 0
    for c in e:
        px = c[0]*math.cos(c[2])
        py = c[0]*math.sin(c[2])
        pz = c[0]*math.sinh(c[1])        
        px_tot += px
        py_tot += py
        pz_tot += pz
        en_tot += math.sqrt(px*px + py*py + pz*pz)
    tmp = en_tot*en_tot - px_tot*px_tot - py_tot*py_tot - pz_tot*pz_tot
    if tmp < 0.0001: 
#         print(tmp)
        tmp = 0
    masses.append(math.sqrt(tmp))
    i+=1
masses = np.array(masses)
masses_over_pt = masses/pttot
norm_masses_over_pt = (masses_over_pt-np.mean(masses_over_pt))/np.std(masses_over_pt)

##### linear regression on EFPs

In [ ]:
for EFP_dim in [1,5,15,25,35,45,53]:
#     EFP_dim is the number of EFP included
    trans_res = norm_results[:EFP_dim].transpose()

    reg = linear_model.LinearRegression()
#     Performing linear regression here
    reg.fit(trans_res, norm_masses_over_pt)
#     the coefficient of each EFP
    w = np.array(reg.coef_)
    
    mass_efp = []
    for i in range(len(trans_res)):
#         calculating the masses using the 53 EFPs
        mass_efp.append((w*trans_res[i]).sum())

#     Place the code to draw the histogram here 

##### linear regression on PCs

In [ ]:
events_pca = []
trans_res = norm_results[:53].transpose()
# pca_comp contains the pc represented by the basis of 53 EFPs
pca_comp = pca.components_
for i in range(len(trans_res)):
    pca_tmp = []
    for j in range(len(pca_comp)):
#         calculate each jet's projection onto the PCs
        pca_tmp.append(np.dot(trans_res[i], pca_comp[j]))
    events_pca.append(pca_tmp)
events_pca = np.array(events_pca)
reg_pca = linear_model.LinearRegression()
reg_pca.fit(events_pca, norm_masses_over_pt)
mass_pca = []
w = np.array(reg_pca.coef_)
for i in range(len(events_pca)):
    mass_pca.append((w*events_pca[i]).sum())

# Place the code to draw the histogram here 

##### The process are the same for other variables

#### Girth

In [ ]:
def theta_diff(t1,t2):
    dphi = abs(t1-t2)
    if(dphi >= math.pi):
        dphi = 2*math.pi - dphi
    return dphi

In [ ]:
eta = []
phi = []
for e in t:
    if e.jetm > 0:
        n = e.jetm
        if e.pt > 20: 
            ev = 1
            eta.append(e.eta)
            phi.append(e.phi)
        else: ev = 0
    elif n >= 0:
        n -= 1
eta = np.array(eta)
phi = np.array(phi)

In [ ]:
girths = []
i = 0
for e in events:
    g = 0
    eta_jet = eta[i]
    phi_jet = phi[i]
    for c in e:
        eta_i = c[1]
        phi_i = c[2]
        g+=c[0]*math.sqrt(theta_diff(eta_i, eta_jet)**2 + theta_diff(phi_i , phi_jet)**2)      
        if math.sqrt(theta_diff(eta_i, eta_jet)**2 + theta_diff(phi_i, phi_jet)**2) > 0.5:
            print
            print(math.sqrt(theta_diff(eta_i, eta_jet)**2 + (phi_i - phi_jet)**2))
            print eta_i, eta_jet, phi_i, phi_jet
        if g > pttot[i]:
            print g, pttot[i]
    g /= pttot[i]
    girths.append(g)
    i+=1

#### Integrated Jet Shape $\Psi(0.1)$

In [ ]:
IJS = []
yes = no = 0
i = 0
for e in events:
    psi = 0
    eta_jet = eta[i]
    phi_jet = phi[i]
    for c in e:
        eta_i = c[1]
        phi_i = c[2]
        r = math.sqrt(theta_diff(eta_i, eta_jet)**2 + theta_diff(phi_i, phi_jet)**2)
#         if r > 0.05: no += 1
        if r <= 0.1:
            yes += 1
            psi += c[0]
        else:
            no += 1
    psi /= pttot[i]
    IJS.append(psi)
    i+=1